In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nbeats_keras.model import NBeatsNet as NBeatsKeras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from nbeats_pytorch.model import NBeatsNet as NBeatsPytorch
from keras.optimizers import RMSprop, Adam
import time
from keras.models import load_model
#from target_data_electronic70_7 import target_X, target_y ,test_X, test_y
#from m4databasis21_7 import base_domain,zt_in,zt_out,M4Meta,inputsize,train_12,train_12_y
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
from tensorflow.keras.losses import Loss
import tensorflow as tf
#from m4databasis35_7_70_7 import train_35,train_35_y,train_70,train_70_y
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Add, Concatenate,Flatten,Reshape
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Lambda
from tensorflow.keras.callbacks import EarlyStopping

2024-10-11 16:57:15.808487: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-11 16:57:15.964472: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-11 16:57:15.964538: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-10-11 16:57:16.643953: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [2]:
target_X= pd.read_csv("../data/ele_train_input_7.csv").iloc[:,1:].values.astype(np.float32) / 10000
target_y =pd.read_csv("../data/ele_train_output_7.csv").iloc[:,1:].values.astype(np.float32)/ 10000
test_X= pd.read_csv("../data/ele_val_input_7.csv").iloc[:,1:].values.astype(np.float32)/ 10000
test_y =pd.read_csv("../data/ele_val_output_7.csv").iloc[:,1:].values.astype(np.float32)/ 10000
X_train=target_X.astype(np.float32)
y_train=target_y.astype(np.float32)
#y_train.astype(np.float32)
backcast_length = X_train.shape[1]
forecast_length = y_train.shape[1]

In [3]:
#################################################################################
# loss SMAPE
class SMAPE(Loss):
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 예측 값의 차원을 맞춤
       # y_pred=tf.clip_by_value(y_pred, 1e-10, tf.reduce_max(y_pred))
       # y_true = tf.clip_by_value(y_true, 1e-10, tf.reduce_max(y_true))
        
        numerator = 100 * tf.abs(y_true- y_pred )
        denominator =  (tf.abs(y_true ) + tf.abs(y_pred))/2
        smape =  numerator /  denominator #tf.clip_by_value(denominator, 1e-10, tf.reduce_max(denominator))
        return tf.reduce_mean(smape)

#################################################################################
# loss MASE
class MASE(Loss):
    def __init__(self, training_data, period, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.scale = self.calculate_scale(training_data, period)
    def seasonal_diff(data, period):
        return data[period:] - data[:-period]

    def calculate_scale(self, training_data, period):
        # 주기 차분 계산
        diff = seasonal_diff(training_data, period)
        scale = np.mean(np.abs(diff))
        return scale
    
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 차원 맞추기
        error = tf.abs(y_true - y_pred)
        return tf.reduce_mean(error / self.scale)

def seasonal_diff(data, period):
    return data[period:] - data[:-period]
#################################################################################
# 하이퍼파라미터 인자 설정
def hyperparameter():
    # 1 backcast
    # 2 forecast
    # 3 inputdim
    # 4 outputdim
    # 5 unit
    # 6 bacth size
    return X_train.shape[1],1,y_train.shape[1]

#################################################################################
# nbeats 모델 생성 함수
def build_model(input_timesteps,features,output_timesteps):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(input_timesteps, features)))
    model.add(LSTM(64, return_sequences=True))
    #model.add(LSTM(unit, return_sequences=True))
    # Use Lambda layer to select the last 'output_timesteps' outputs
    model.add(Lambda(lambda x: x[:, -24:, :]))
    model.add(TimeDistributed(Dense(1)))
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer=optimizer)
    return model

#################################################################################
# 부트스트랩 샘플링
# 배깅

def train_bagging_models(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    input_timesteps,features,output_timesteps= hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = build_model(input_timesteps,features,output_timesteps)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        #select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        #X_bootstrap = X_train[select]
        #y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 1, restore_best_weights=True)
        history = model.fit(X_train, y_train, batch_size = batch_size_,
                  epochs=epochs_, verbose=1, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys

#################################################################################
# SMAPE 용
def train_bagging_models_smape(num_models, loss_fn , epochs_, patience_,batch_size_):
    models = {}
    backcast,forecast,in_dim,out_dim,unit = hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = bulid_model(backcast,forecast,in_dim,out_dim)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=0.001)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        #select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        #_bootstrap = X_train[select]
        #y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 1, restore_best_weights=True)
        history = model.fit(X_train, y_train, batch_size = batch_size_,
                  epochs=epochs_, verbose=1, 
                  callbacks=[early_stop],
                 validation_split = 0.2)
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys



#################################################################################

def bagging_predict(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return np.median(predictions, axis=0)

def bagging_predict2(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return predictions

In [4]:
model_num = 5


mase_models = train_bagging_models(model_num, MASE(y_train,24),2000,10,8,0.0001)
mape_models = train_bagging_models(model_num,'mape',2000,10,8,0.0001)
smape_models = train_bagging_models(model_num, SMAPE(),2000,10,8,0.0001)
mae_models = train_bagging_models(model_num, 'mae',2000,10,8,0.0001)
mse_models = train_bagging_models(model_num, 'mse',2000,10,8,0.0001)

2024-10-11 16:57:23.348076: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-10-11 16:57:23.348132: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ymlee2-desktop): /proc/driver/nvidia/version does not exist
2024-10-11 16:57:23.349580: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/2000
73/73 [==============================] - 11s 111ms/step - loss: 4.1611 - val_loss: 1.1223
Epoch 2/2000
73/73 [==============================] - 8s 110ms/step - loss: 0.9187 - val_loss: 0.9334
Epoch 3/2000
73/73 [==============================] - 8s 108ms/step - loss: 0.8938 - val_loss: 0.9168
Epoch 4/2000
73/73 [==============================] - 7s 101ms/step - loss: 0.8854 - val_loss: 0.9246
Epoch 5/2000
73/73 [==============================] - 8s 109ms/step - loss: 0.8619 - val_loss: 0.8749
Epoch 6/2000
73/73 [==============================] - 7s 99ms/step - loss: 0.8300 - val_loss: 0.8174
Epoch 7/2000
73/73 [==============================] - 7s 98ms/step - loss: 0.8012 - val_loss: 0.8040
Epoch 8/2000
73/73 [==============================] - 8s 108ms/step - loss: 0.7636 - val_loss: 0.7397
Epoch 9/2000
73/73 [==============================] - 8s 109ms/step - loss: 0.7156 - val_loss: 0.6704
Epoch 10/2000
73/73 [==============================] - 8s 103ms/step - loss: 0.6507

In [5]:
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models
pred4,_=mae_models
pred5,_=mse_models

smape_predictions = bagging_predict2(pred1, test_X)
mase_predictions = bagging_predict2(pred2, test_X)
mape_predictions = bagging_predict2(pred3, test_X)
mae_predictions = bagging_predict2(pred4, test_X)
mse_predictions = bagging_predict2(pred5, test_X)

12/12 [==============================] - 1s 31ms/step


In [13]:
#pd.DataFrame(fin_pred.reshape(-1,24)).to_csv("pred/lstm.csv")

In [6]:
concat = np.concatenate([smape_predictions, mase_predictions,mape_predictions,mae_predictions,mse_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
pd.DataFrame(fin_pred.flatten()).to_csv("result7/lstm.csv")

np.sqrt(mean_squared_error(test_y.flatten(),fin_pred.flatten())),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

(0.34226954, 0.22443897)

In [7]:

concat = np.concatenate([mae_predictions, mase_predictions,mse_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
pd.DataFrame(fin_pred.flatten()).to_csv("best7/LSTM.csv")

np.sqrt(mean_squared_error(test_y.flatten(),fin_pred.flatten())),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

(0.34471175, 0.22660251)

In [8]:

concat = np.concatenate([smape_predictions, mase_predictions,mape_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred.flatten())),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

(0.34499446, 0.22593692)

In [9]:

concat = np.concatenate([smape_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred.flatten())),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

(0.38135096, 0.24693182)

In [10]:
concat = np.concatenate([mape_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred.flatten())),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

(0.3615845, 0.2373837)

In [11]:
concat = np.concatenate([mase_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred.flatten())),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

(0.3558516, 0.23579495)

In [12]:
concat = np.concatenate([mae_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred.flatten())),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

(0.3508729, 0.2302761)

In [13]:
concat = np.concatenate([mse_predictions],axis=0)
fin_pred = np.median(concat,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
np.sqrt(mean_squared_error(test_y.flatten(),fin_pred.flatten())),mean_absolute_error(test_y.flatten(),fin_pred.flatten())

(0.35632575, 0.24217606)

In [17]:
concat_mase = np.concatenate([np.nan_to_num(np.array(mase_predictions), nan=0)])
fin_pred_mase = np.median(concat_mase,axis=0)
MASE= np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_mase .flatten())).round(5)

concat_mape = np.concatenate([np.nan_to_num(np.array(mape_predictions), nan=0)])
fin_pred_mape = np.median(concat_mape,axis=0)
MAPE= np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_mape .flatten())).round(5)

concat_smape = np.concatenate([np.nan_to_num(np.array(smape_predictions), nan=0)])
fin_pred_smape = np.median(concat_smape,axis=0)
sMAPE= np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_smape .flatten())).round(5)

concat_mae = np.concatenate([np.nan_to_num(np.array(mae_predictions), nan=0)])
fin_pred_mae = np.median(concat_mae,axis=0)
MAE= np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_mae .flatten())).round(5)

concat_mse = np.concatenate([np.nan_to_num(np.array(mse_predictions), nan=0)])
fin_pred_mse = np.median(concat_mse,axis=0)
MSE= np.sqrt(mean_squared_error(test_y.flatten(),fin_pred_mse .flatten())).round(5)

performance = np.array([MASE, MAPE,sMAPE,MAE,MSE])
beta = 1  # 조정 파라미터
weights = np.exp(-beta * performance)

gd= np.concatenate([fin_pred_mase.flatten().reshape(1,-1),
                    fin_pred_mape.flatten().reshape(1,-1),
                   fin_pred_smape.flatten().reshape(1,-1),
                   fin_pred_mae.flatten().reshape(1,-1),
                   fin_pred_mse.flatten().reshape(1,-1)],axis=0)

normalized_weights = weights / np.sum(weights)

# 각 모델의 예측값에 가중치를 부여하여 앙상블 예측 생성
ensemble_prediction = np.dot(normalized_weights, gd)
print(np.sqrt(mean_squared_error(test_y.flatten(),ensemble_prediction)).round(5))

0.34584


In [18]:
pd.DataFrame(ensemble_prediction).to_csv("exp7/vanila_LSTM.csv")